In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms.functional as TF
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, Dataset
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from tqdm.auto import tqdm

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

torch.manual_seed(0)
np.random.seed(0)

Using device: cuda


# 1. Change Point Detection

Data Generation

$t = 0 → s(0) = [x_{00}, x_{01}, ..., x_{09}] \\
t = 1 → s(1) = [x_{10}, x_{11}, ..., x_{19}] \\
t = 2 → s(2) = [x_{20}, x_{21}, ..., x_{29}] \\
\cdots \\
t = 99 → s(99)$

In [10]:
def generate_sequence(max_T=100):
    length = max_T

    # Decide change point including "no change"
    T = random.randint(0, length)  

    # Create the full sequence of shape (length, 10)
    seq = torch.randn(length, 10)

    # Pick 5 indices out of 10
    indices = random.sample(range(10), 5)

    # Create means for these indices
    mus = {idx: random.uniform(-1, 1) for idx in indices}

    # Apply shift only after change point
    if T < length:  
        for idx in indices:
            seq[T:, idx] += mus[idx]

    # Label: 0 if no change happened yet, 1 if change occurred
    label = 1 if T < length else 0

    return seq, label, T


In [31]:
class ChangePointDataset(Dataset):
    def __init__(self, num_sequences, device='cpu'):
        self.num_sequences = num_sequences
        self.device = device

    def __len__(self):
        return self.num_sequences

    def __getitem__(self, idx):
        seq, label, T = generate_sequence()
        seq = seq.to(self.device)
        labels = torch.zeros(seq.shape[0], 1, device=self.device)
        labels[T:] = 1
        return seq, labels

Get dataset

In [38]:
dataset = ChangePointDataset(500, device=device)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

RNN and LSTM models

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size=10, hidden_size=20, output_size=1, device='cpu'):
        super(RNNModel, self).__init__()

        self.rnn = nn.RNN(input_size, hidden_size, num_layers=1, bias=True, 
                          dropout=0, bidirectional=False, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, output_size)

        self.to(device)

    def forward(self, x):
        # input x: (batch_size, time, features)
        out, _ = self.rnn(x) # out: (batch_size, time, hidden_size)
        logits = self.fc(out) # logits: (batch_size, time, 1)
        probability = torch.sigmoid(logits) # probability: (batch_size, time, 1)

        return probability

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size=10, hidden_size=20, output_size=1, device='cpu'):
        super(LSTMModel, self).__init__()

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=1, bias=True, 
                          dropout=0, bidirectional=False, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, output_size)

        self.to(device)

    def forward(self, x):
        # input x: (batch_size, time, features)
        out, _ = self.lstm(x) # out: (batch_size, time, hidden_size)
        logits = self.fc(out) # logits: (batch_size, time, 1)
        probability = torch.sigmoid(logits) # probability: (batch_size, time, 1)

        return probability

In [17]:
def get_probabilities(model, seq):
    model.eval()
    with torch.no_grad():
        seq = seq.unsqueeze(0)          # (1, 100, 10)
        prob = model(seq).squeeze()     # (100,)
    return prob

In [34]:
def train_model(model, dataloader, num_epochs=10, lr=0.01):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()

    loss_history = []

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for sequences, labels in dataloader:
            optimizer.zero_grad()

            outputs = model(sequences)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item() * sequences.size(0)
            loss_history.append(epoch_loss)
        epoch_loss /= len(dataloader.dataset)

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
    
    return model, loss_history

In [35]:
rnn = RNNModel(input_size=10, hidden_size=20, output_size=1, device=device)
rnn, loss_history = train_model(rnn, dataloader, num_epochs=10, lr=0.001)

Epoch 1/10, Loss: 0.6951
Epoch 2/10, Loss: 0.6988
Epoch 3/10, Loss: 0.6955
Epoch 4/10, Loss: 0.7015
Epoch 5/10, Loss: 0.6991
Epoch 6/10, Loss: 0.6990
Epoch 7/10, Loss: 0.6954
Epoch 8/10, Loss: 0.6936
Epoch 9/10, Loss: 0.6961
Epoch 10/10, Loss: 0.6941


In [36]:
get_probabilities(rnn, dataloader.dataset[0][0])

tensor([0.4777, 0.5229, 0.5026, 0.5077, 0.5181, 0.4574, 0.4967, 0.4999, 0.4509,
        0.5031, 0.4791, 0.4900, 0.4999, 0.5276, 0.5282, 0.5377, 0.4642, 0.5023,
        0.4614, 0.5072, 0.5016, 0.5270, 0.5088, 0.5008, 0.4616, 0.5024, 0.4895,
        0.5322, 0.5273, 0.5359, 0.5229, 0.4994, 0.5014, 0.4537, 0.4936, 0.4755,
        0.4636, 0.4961, 0.4460, 0.4849, 0.5080, 0.4947, 0.4589, 0.4983, 0.5175,
        0.4833, 0.4931, 0.4592, 0.5320, 0.5017, 0.4844, 0.5195, 0.4847, 0.5064,
        0.5012, 0.4608, 0.4646, 0.5693, 0.5280, 0.5093, 0.5065, 0.4849, 0.5111,
        0.4813, 0.5102, 0.5154, 0.4806, 0.4729, 0.4997, 0.4689, 0.4847, 0.5011,
        0.5016, 0.4753, 0.4948, 0.5390, 0.5541, 0.5377, 0.5426, 0.5418, 0.4745,
        0.5163, 0.4943, 0.5013, 0.5079, 0.5229, 0.5470, 0.5502, 0.5578, 0.5173,
        0.5279, 0.5188, 0.5168, 0.5677, 0.5232, 0.5387, 0.5406, 0.5140, 0.5264,
        0.5182], device='cuda:0')